# Notebook for End to End testing in the MID PSI

###### Last updated 01/08/24

This notebook is used to execute an end-to-end scan using the following Mid Products: Dish LMC, SPFRx, TMC, CSP.LMC, CBF, and SDP. It also provides the option to run multiple offset scan to enable calibration, following the [Five Point Calibration Scan Controls notebook](https://gitlab.com/ska-telescope/ska-jupyter-scripting/-/blob/main/notebooks/observing/MID_five_point_calibration_scan_controls.ipynb?ref_type=heads).

To use Dish LMC/SPFRx, ensure the namespace to use with this project is started with `DISH_LMC_ENABLED` set to `true`. Otherwise, the default setting of `false` can be used. 

If using SPFRx, only Talon1 can be used (as it is what the spfrx is connected to) and the rxpu must be signed out.

### For using BITE

BITE Functionality has been spun out to the [bite_generation notebook](bite_generation.ipynb), which should be referred to when needed. These steps are labeled as BITE STEP and are:
- Setting variables
- Loading Config Data
- Generating BITE Data
- Starting LSTV Replay
- Stopping LSTV Replay

## 1 Setup

### Deployment checks

On the bootup of the namespace, once all all the pods are in the running state, check each of the dishleafnode logs for the following error:

<div class="alert alert-block alert-danger">
ValueError: StationaryBody needs a location to calculate coordinates - did you specify an Antenna?
</div>

If any of the dishleafnode pods show this error then:
1. Delete the dishleafnode pod
2. Monitor the logs again after re-creation and ensure the same error doesn't show up again
3. If it does then repeat steps 1 and 2, eventually it should work

### 1.1 Environment Setup

Start by importing all the libraries needed for this notebook

In [1]:
import sys

sys.path.append("../../src")

import json
import os
import time
from time import localtime, sleep, strftime

from IPython.display import clear_output
from tango import Database, DevFailed, DeviceProxy

import notebook_tools.generate_fsp as generate_fsp
import notebook_tools.wait_for_tango as wait_for_tango

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [ ]:
!kubectl get ns | grep ska-mid-psi

Now, use this to set the namespace the notebook will use.

In [2]:
psi_namespace = ""  # Namespace to be used
using_spfrx = False  # Set to True if using spfrx, otherwise set to False if using BITE
simulation_mode = False
target_boards = [
    1, 2, 3, 4
]  # Talon board(s) to use for the notebook. 

# NOTE: As of Jan 29, 2025
# If using SPFRx, only talon 5 can be used.
# If doing 4-receptor AA0.5 testing, use talons 1-4 or talons 9/10/15/16. 
# Talons 5-8 can be used individually for AA0.5 testing but not together for 4-receptor AA0.5 testing.

FSP calculation parameters

In [ ]:
start_freq = 350000000  # Start frequency in Hz
num_fsps_available = len(target_boards)
print(
    f"Maximum end frequency based on params above: {generate_fsp.calculate_end_freq(start_freq, num_fsps_available)}"
)
print("NOTE: MAX FREQ LIMIT for BAND 1/2 is 1760000000")

In [4]:
end_freq = 500000000  # Replace this number based on calculation in block above
if end_freq > 1760000000:
    raise Exception("Specified End Frequency is higher than max allowed")

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<a id='setting bite vars'></a>
<div class="alert">
    <h3>(BITE STEP) Set Variables in the BITE Notebook</h3>
    <br>
    If running the BITE notebook, at this point ensure the variables for namespace and test_id are properly set.
</div>
</body>

Now, load in the other variables this notebook will use, along with config files to pass in. These vars should not need to be changed in most use cases.

In [ ]:
TANGO_HOST = f"databaseds-tango-base.{psi_namespace}.svc.cluster.local:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"

# TMC config files
ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources_psi.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan_psi.json"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{TMC_CONFIGS}/release_resources.json"

# For running offset scans
SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

# CBF dish files
CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"
HW_CONFIG_FOLDER = os.path.join(CBF_CONFIGS, "hw_config")
INIT_SYS_PARAM_FILE = os.path.join(CBF_CONFIGS, "sys_params/initial_system_param_psi.json")

# Select HW file based on boards selected
if target_boards[0] >= 13:
    print("Using HW Config File for Talons 13-16")
    hw_config = "hw_config_psi_13_16.yaml"
if target_boards[0] >= 9:
    print("Using HW Config File for Talons 9-12")
    hw_config = "hw_config_psi_9_12.yaml"  
elif target_boards[0] >= 5:
    print("Using HW Config File for Talons 5-8")
    hw_config = "hw_config_psi_5_8.yaml"
elif target_boards[0] >= 1:
    print("Using HW Config File for Talons 1-4")
    hw_config = "hw_config_psi_1_4.yaml"

if target_boards[0] >=5:
    print("Mapping talons of higher numbers to 1-4")
    target_boards = list(map(lambda x: x - (((x - 1) // 4) * 4), target_boards))

HW_CONFIG_FILE = os.path.join(HW_CONFIG_FOLDER, hw_config)
!kubectl cp $HW_CONFIG_FILE $psi_namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

# Check files can be reached.
files = [
    HW_CONFIG_FILE,
    INIT_SYS_PARAM_FILE,
    DISH_CONFIG_FILE,
    ASSIGN_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    RELEASE_RESOURCES_FILE,
]

# Slim config files setup
# Load in nothing if using 1 board as not needed, load in 4vcc 1fsp if needed
print("Checking SLIM configs:")
if len(target_boards) == 1:
    slim_fs_config = ""
    slim_vis_config = ""
elif len(target_boards) == 2:
    slim_fs_config = "fs_slim_2vcc_2fsp.yaml"  # update if necessary
    slim_vis_config = "vis_slim_2fsp_1vis.yaml"  # update if necessary
elif len(target_boards) == 4:
    slim_fs_config = "fs_slim_4vcc_4fsp.yaml"  # update if necessary
    slim_vis_config = "vis_slim_4fsp_1vis.yaml"  # update if necessary

SLIM_CONFIGS = os.path.join(CBF_CONFIGS, "slim_config")
SLIM_FS_CONFIG_FILE = os.path.join(SLIM_CONFIGS, slim_fs_config)
SLIM_VIS_CONFIG_FILE = os.path.join(SLIM_CONFIGS, slim_vis_config)

if slim_fs_config != "":
    print(f"    Loading custom SLIM fs config: {slim_fs_config}")
    !kubectl cp $SLIM_FS_CONFIG_FILE $psi_namespace/ds-cbfcontroller-controller-0:/app/mnt/slim/fs/slim_config.yaml
    files.append(SLIM_FS_CONFIG_FILE)
else:
    print(f"    SLIM fs will use default config: {slim_fs_config}")

if slim_vis_config != "":
    print(f"    Loading custom SLIM vis config: {slim_vis_config}")
    !kubectl cp $SLIM_VIS_CONFIG_FILE $psi_namespace/ds-cbfcontroller-controller-0:/app/mnt/slim/vis/slim_config.yaml
    files.append(SLIM_VIS_CONFIG_FILE)
else:
    print(f"    SLIM vis will use default config: {slim_vis_config}")

# Ensure the files exist
print("Checking to ensure files exist:")
for file in files:
    if os.path.isfile(file):
        print(f"    {file} exists: ✔️")
    else:
        print(f"    {file} does not exist ❌")

# Map the talon boards to receptor IDs
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]
RECEPTORS = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards))

# Sanity check to ensure that Receptors = talons match
if len(RECEPTORS) == len(target_boards):
    print("Receptors match number of talons:")
else:
    print("ERROR: Receptor/talon mismatch: ")

print(f"    Receptors: {RECEPTORS}")
print(f"    Talons: {target_boards}")

With the namespace set, the useful front ends to monitor the behaviour of the system can be accessed using the following URLs.

In [ ]:
print("For showing the signal display output:")
print(f"https://142.73.34.170/{psi_namespace}/signal/display/")

print("\nTaranta devices for checking all the TANGO devices currently in the namespace:")
print(f"https://142.73.34.170/{psi_namespace}/taranta/devices")

print("\nTelescope Monitoring Dashboard:")
print(
    f"https://142.73.34.170/{psi_namespace}/taranta/dashboard?id=669ea8d82bc4790019e64b27&mode=run"
)

print("\nMid CBF Overview Dashboard:")
print(
    f"https://142.73.34.170/{psi_namespace}/taranta/dashboard?id=666cb28b5e5d4f0012197e5f&mode=run"
)

print("\nFor using the EDA configurator (Only if SKA_TANGO_ARCHIVER=true for the namespace):")
print(f"https://142.73.34.170/{psi_namespace}/configurator/")

### 1.3 Setup Device Proxies

First, to interact with the devices used by this notebook, TANGO device proxies must be set up to connect to and control them.

In [ ]:
# TMC Proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

# CSP Proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray = DeviceProxy("mid-csp/subarray/01")

# CBF Proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# SDP Proxies
sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

# Leaf Node Proxies
csp_subarray_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
sdp_subarray_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/sdp_subarray01")

# Deployer Proxy
deployer = DeviceProxy("mid_csp_cbf/ec/deployer")

# print the states of each
devices = [
    tmc_central_node,
    tmc_csp_master,
    tmc_subarray,
    csp_subarray_leaf_node,
    sdp_subarray_leaf_node,
    csp_control,
    csp_subarray,
    cbf_controller,
    cbf_subarray,
    sdp_subarray,
]

for receptor in RECEPTORS:
    dish_leaf_node_idx = "0" + receptor[3:]
    dish_leaf_node_fqdn = f"ska_mid/tm_leaf_node/d{dish_leaf_node_idx}"
    dish_leaf_node_dp = DeviceProxy(dish_leaf_node_fqdn)
    devices.append(dish_leaf_node_dp)
    
for device in devices:
    padding = "-" * (40 - len(device.dev_name()))
    print(f"{device.dev_name()}'s state {padding}> {device.state()}")

### 1.4 MCS Deployer Setup and Download Artifacts

To use the deployer, set the dish ID that will be deployed to.

In [ ]:
db = Database()
deployer.targetTalons = target_boards
print("Deployer will target the following talons:", deployer.targetTalons)
deployer.generate_config_jsons()

Now the actual download step can be run, this will take some time.

In [10]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(10000)

Once the downloaded, the TANGO device database can be configured with the new downloads

In [11]:
deployer.configure_db()

If desired, the devices can be checked to ensure they have been downloaded.

In [ ]:
print(*db.get_device_exported("*").value_string, sep="\n")

## 2 Set Up Devices

### 2.1 Setting up the CSP/CBF

With the connection established to the devices, set the admin and simulation mode to both be 0. This will allow the running of commands and ensure real hardware is being used.

In [12]:
# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = simulation_mode
csp_control.commandTimeout = 99
csp_subarray.commandTimeout = 99

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

In [ ]:
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Controller Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Subarray Simulation Mode: {bool(cbf_subarray.simulationMode)}")
print(f"  CBF Controller Timeout: {csp_control.commandTimeout} sec")
print(f"  CSP Subarray Timeout: {csp_subarray.commandTimeout} sec")

print("\nChecking admin mode after setting to ONLINE:")
print(f"  CSP Control: {csp_control.adminMode.name}")
print(f"  CSP Subarray: {csp_subarray.adminMode.name}")
print(f"  CBF Controller: {cbf_controller.adminMode.name}")
print(f"  CBF Subarray: {cbf_subarray.adminMode.name}")

### 2.2 Load the Dish Vcc Config / Init Sys Params

Next, load in the dish config file to the central node:

In [ ]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

# Reach out to grab the tagged Telescope model with K-value of 1
# See: https://gitlab.com/ska-telescope/ska-telmodel-data/-/tree/0.1.0-rc-mid-itf/tmdata/instrument/ska1_mid_itf
dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

# Actually load in the dish config
print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

# Wait for dishvcc to be loaded...
wait_seconds = 0
while not tmc_central_node.isDishVccConfigSet:
    clear_output(wait=True)
    print(f"Waiting for DishVCC to be set, {wait_seconds} seconds elapsed...")
    sleep(2)
    wait_seconds += 2

clear_output(wait=True)
print("DishVCC has been set!")

In [ ]:
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Load Config Data</h3>
    <br>
    Now, if required, the BITE configuration data can be loaded in using the BITE notebook.
</div>
</body>

### 2.3 Turn the Telescope On

Now, turn the telescope itself on:

In [ ]:
print("Running the TelescopeOn command")
tmc_central_node.set_timeout_millis(100000)
tmc_central_node.TelescopeOn()

startup_time = 0
alert_msg = ""
while int(tmc_central_node.telescopeState) != 0:
    print(f"\r Telescope is starting up, {startup_time} seconds elapsed. {alert_msg}", end="")
    sleep(5)
    startup_time += 5
    if startup_time > 120:
        alert_msg = "Startup is taking longer than expected, try running LRU power off scripts."
print(f"\n Telescope has started after {startup_time} seconds.")

#### 2.3.1 Remedying Telescope on 

In some cases, the steps for powering on will not complete, due to DDR calibration failing, causing the HPS to error out. This can be checked by monitoring the CBF controller and logconsumer to see if a HPS error has occurred. If monitoring central node while running, these errors will likely surface as timeout errors when running the above step
Check if these error messages occur: 

- hpsmaster (via ds-talonlogconsumer device): 
   - `DsHpsMaster::configure: Timeout waiting for Talon Status`

- ds-cbfcontroller-controller device: 
   - `Configure command for talondx-.../hpsmaster/hps-2 device failed with error code 4`
   - `Failed to configure Talon boards`
   - `Exiting command OnCommand with return_code ResultCode.FAILED, message: 'Failed to configure Talon boards'.`

If this is the case, the LRU will have to be powered down, then powered on using the scripts available through talon_power_lru.sh. **Note that this will require that both boards on each LRU (like 1 and 2) will be shut off**. See [CIP-2344](https://jira.skatelescope.org/browse/CIP-2344) for more details.

If it is required to reset the LRU, on a dev machine run the following steps:

***

Once the On command has been run successfully, check that the Telescope State is 0 and that states of the other devices are ON.

In [ ]:
print("Verifying the states:")
print(f"  TMC Central Node Telescope State: {int(tmc_central_node.TelescopeState)}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")
print(f"  TMC Subarray State: {tmc_subarray.State()}")

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Generate BITE Data</h3>
    <br>
    With the telescope on, the section of the BITE generation notebook that handles the actual generation of BITE data can be run.
</div>
</body>

### 2.4 Assign Resources to the Telescope

Start assigning resources by setting up the JSON file as needed:

In [ ]:
os.environ["TZ"] = "America/Vancouver"
time_now = localtime()
date = strftime("%Y%m%d", time_now)
time_now = strftime("%H%M%S", time_now)
eb_id = f"eb-test-{date}-{time_now}"
pb_id = f"pb-test-{date}-{time_now}"

channel_count = generate_fsp.calculate_channel_count(start_freq, end_freq)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS
    assign_resources_json["sdp"]["execution_block"]["eb_id"] = eb_id
    assign_resources_json["sdp"]["processing_blocks"][0]["pb_id"] = pb_id

    assign_resources_json["sdp"]["execution_block"]["channels"][0]["spectral_windows"][0][
        "freq_min"
    ] = float(start_freq)
    assign_resources_json["sdp"]["execution_block"]["channels"][0]["spectral_windows"][0][
        "freq_max"
    ] = float(end_freq)
    assign_resources_json["sdp"]["execution_block"]["channels"][0]["spectral_windows"][0][
        "count"
    ] = channel_count

print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

Now, the command to actually assign the resources can be run, and the TMC should go to idle (2)

In [ ]:
tmc_subarray.AssignResources(json.dumps(assign_resources_json))
wait_for_tango.wait_for_state(tmc_subarray, 2)

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Start LSTV Replay</h3>
    <br>
    If using BITE for data stream generation, the respective step in the BITE notebook can be run at this point.
</div>
</body>

## 3 Running the Scan(s)

With everything set up, the scans can now be run:

### 3.1 Configure Scan

<div class="alert alert-block alert-danger">
During the configure scan there is still a possibility of TMC getting stuck in <b>CONFIGURING</b> due to a network issue in the PSI. If you see the following error in the ds-cspsubarrayleafnode-01-0 pod then you will have to redeploy your namespace and try again:

AttributeError: 'NoneType' object has no attribute 'delays'
</div>

Before running the scan configuration, ensure that the SDP vis pod has spun up:

In [ ]:
!kubectl -n $psi_namespace-sdp get pods | grep vis-receive

Now, append the configure scan JSON file as needed before uploading it.

In [ ]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

# Create and Append FSPs to scan config file
# note that channel offset will change with ADR-99
configure_scan_json["csp"]["midcbf"]["correlation"]["processing_regions"][0]["fsp_ids"] = []
fsp_list = generate_fsp.generate_fsp_list(start_freq, end_freq, target_boards)
configure_scan_json["csp"]["midcbf"]["correlation"]["processing_regions"][0]["fsp_ids"] = fsp_list
configure_scan_json["csp"]["midcbf"]["correlation"]["processing_regions"][0][
    "start_freq"
] = start_freq
configure_scan_json["csp"]["midcbf"]["correlation"]["processing_regions"][0][
    "channel_count"
] = channel_count

# Assign the config ID
configure_scan_json["csp"]["common"][
    "config_id"
] = f"{len(target_boards)} receptor, band 1, {len(fsp_list)} FSP(s), no options"

print("Appended configure scan file:")
print(json.dumps(configure_scan_json, indent=2))

Send the configuration, and wait for the TMC to go to ready:

In [ ]:
tmc_subarray.Configure(json.dumps(configure_scan_json))
wait_for_tango.wait_for_state(tmc_subarray, 4)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

Optional: 

Pointing state expected value: TRACK

Dish Mode expected value: OPERATE

In [ ]:
for receptor in RECEPTORS:
    print(f"Dish Leaf Node {dish_leaf_node_idx} Pointing State: {dish_leaf_node_dp.pointingState.name}")
    print(f"Dish Leaf Node {dish_leaf_node_idx} Dish Mode: {dish_leaf_node_dp.DishMode.name}\n")

### 3.2 Running a Non-Offset Scan

Now, the scan itself can be run by sending the command to the TMC:

In [ ]:
print("Running the Scan command: subarray obsstate should go to Scanning (5)")

with open(SCAN_FILE, encoding="utf-8") as f:
    scan_json = f.read()

print(f"\nscan_json file contents: \n{scan_json}")

tmc_subarray.Scan(scan_json)
wait_for_tango.wait_for_state(cbf_subarray, 5)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState.name}")

Monitor the SDP vis pod and the signal display webpage to ensure the scan is underway.

### 3.3 Ending Initial Scan


After allowing the scan to run for a while, end it via the appropriate command:

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready (4) state")

tmc_subarray.EndScan()
wait_for_tango.wait_for_state(tmc_subarray, 4)
print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState.name}")

After the above cell, wait for the subarray to go to the Ready (4) state.

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState.name}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState.name}\n")

### 3.4 Running Offset Scans for Multi-Point Calibration (Optional)

Now, loop through the offset scans, using the combos to set the scan json. For each, configure it by passing in JSON, then scan again, letting it run before stopping.

In [ ]:
scan_run = 1
for offset in SCAN_COMBOS:
    print(f"setting offset to: {offset}")
    # Configure for this offset scan
    partial_configure_json = {
        "interface": "https://schema.skao.int/ska-tmc-configure/2.2",
        "transaction_id": f"txn-....-0000{scan_run}",
        "scan_id": scan_run,
        "pointing": {"target": {"ca_offset_arcsec": offset[0], "ie_offset_arcsec": offset[1]}},
        "tmc": {"partial_configuration": True},
    }
    print("Partial Config to load:")
    print(json.dumps(partial_configure_json, indent=2))
    print(".......")
    sleep(10)
    tmc_subarray.Configure(json.dumps(partial_configure_json))
    wait_for_tango.wait_for_state(tmc_subarray, 4)

    # Send the Scan command along with relevant JSON,incrementing the scan ID and transaction ID
    partial_scan_json = {
        "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
        "transaction_id": f"txn-....-0000{scan_run}",
        "scan_id": scan_run,
    }
    print(json.dumps(partial_scan_json, indent=2))
    tmc_subarray.Scan(json.dumps(partial_scan_json))
    wait_for_tango.wait_for_state(tmc_subarray, 5)
    # let the scan run for a bit...
    sleep(30)
    # While the scan is running, refresh the signal page and monitor the vis pod logs to ensure the data is coming through.

    # End the scan
    tmc_subarray.EndScan()
    wait_for_tango.wait_for_state(tmc_subarray, 4)

    scan_run += 1
    print("============================")
print("Done offsets!")

## 4 Cleanup

Now that the scan(s) are complete, the namespace and devices can be shut down and cleaned up:

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Stopping LSTV Replay</h3>
    <br>
    Now with the scan(s) done, if using BITE, run the LSTV replay section of the BITE generation notebook.
</div>
</body>

Start by ending any running scans.

In [ ]:
print("Running the End command: subarray obsstate should go to Idle (2) state")

tmc_subarray.End()
wait_for_tango.wait_for_state(tmc_subarray, 2)
print(f"\nTMC Subarray Observation State: {cbf_subarray.obsState.name}")

And check that the subarray has gone to the IDLE (2) state.

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState.name}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState.name}")

### 4.1 Release Resources

Once the scans are stopped and finished, clear the resources from the appropriate devices.

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

tmc_subarray.ReleaseAllResources()
while tmc_subarray.obsState != 0:
    sleep(5)
    clear_output(wait=True)
    print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState.name}")

And ensure that the subarray obsstate goes to EMPTY (0). At this stage, the receptor IDs should also be empty.

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState.name}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState.name}")

### 4.2 Turn the Telescope Off

Finally, send the Off command to the TMC

In [ ]:
print("Running the TelescopeOff command")
tmc_central_node.TelescopeOff()

while int(tmc_central_node.TelescopeState) != 1:
    sleep(5)
    clear_output(wait=True)
    print("These should all go to OFF")
    print(f"TMC Central Node State: {int(tmc_central_node.TelescopeState)}")
    print(f"CSP Control State: {csp_control.State()}")
    print(f"CBF Controller State: {cbf_controller.State()}")